# Generating Induced Hierarchy
Code inspired by Wan, et al: https://arxiv.org/pdf/2004.00221.pdf

In [2]:
import sys
sys.path.insert(0, "../src/util")
sys.path.insert(0, "../src/model")

import argparse
import torchvision
# this is how we'll generate the wnids and hierarchy
from nbdt import data
from nltk.corpus import wordnet as wn
from pathlib import Path
from nbdt.utils import Colors, generate_kwargs

from wn_utils import *
from graph import *
from dir_grab import *

import os

In [ ]:
# first we have to change these class names into wordnet ids, but since
# theyre in the format of class-XXX, nothing important will be gained from 
# this information
classes = os.listdir('/../teams/DSC180A_FA20_A00/a01group09/train_snakes_r1')

# path to write the wnids.txt file to
path = Path("/data/wnids/snakes.txt")

In [ ]:
def synset_to_wnid(synset):
    return f"{synset.pos()}{synset.offset():08d}"

In [ ]:
class FakeSynset:
    '''
    Class to create fake Synonym sets for when wordnet cannot find one
    to use
    '''
    def __init__(self, wnid):
        self.wnid = wnid

        assert isinstance(wnid, str)

    @staticmethod
    def create_from_offset(offset):
        return FakeSynset("f{:08d}".format(offset))

    def offset(self):
        return int(self.wnid[1:])

    def pos(self):
        return "f"

    def name(self):
        return "(generated)"

    def definition(self):
        return "(generated)"

In [ ]:
failures = []
wnids = []

for i, cls in enumerate(classes):
    synsets = wn.synsets(cls, pos=wn.NOUN)
    if not synsets:
        #Colors.red(f"==> Failed to find synset for {cls}. Using fake synset...")
        failures.append(cls)
        synsets = [FakeSynset.create_from_offset(i)]
    synset = synsets[0]
        
    wnid = synset_to_wnid(synset)
    print(f"{wnid}: ({cls}) {synset.definition()}")
    wnids.append(wnid)

In [ ]:
# write to path
with open(str(path), "w") as f:
        f.write("\n".join(wnids))

# Induced Hierarchy
Now that the wnids have been generated, we will build an induced hierarchy from the state_dict of our baseline model

In [ ]:
generate_hierarchy(dataset='snakes', arch='densenet121', model=model, method='random')